<a href="https://colab.research.google.com/github/anhquan-ngo/sc_analytics/blob/main/WebScrawltoChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install Dependencies
# We use sentence-transformers for free local embeddings (no API key needed)
!pip install -q langchain langchain-community faiss-cpu beautifulsoup4 sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# @title 2. Run the Free AI Chatbot
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# The Built-in Colab AI (No API Key needed)
from google.colab import ai

# LangChain & Vector Store components
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader

# --- 1. CRAWLER FUNCTION ---
def crawl_website(start_url, max_pages=15):
    """Crawls the website to find relevant sub-pages."""
    if not start_url.startswith('http'):
        start_url = 'https://' + start_url

    domain = urlparse(start_url).netloc
    visited = set()
    to_visit = [start_url]
    found_urls = []

    print(f"\n🕷️ Crawling {start_url} (Limit: {max_pages} pages)...")

    while to_visit and len(found_urls) < max_pages:
        current_url = to_visit.pop(0)
        if current_url in visited: continue

        try:
            response = requests.get(current_url, timeout=5)
            # Only process HTML
            if response.status_code == 200 and 'text/html' in response.headers.get('Content-Type', ''):
                visited.add(current_url)
                found_urls.append(current_url)
                print(f"Found: {current_url}")

                soup = BeautifulSoup(response.text, 'html.parser')
                for link in soup.find_all('a', href=True):
                    full_url = urljoin(current_url, link['href']).split('#')[0]
                    # Only internal links
                    if urlparse(full_url).netloc == domain:
                        if full_url not in visited and full_url not in to_visit:
                            to_visit.append(full_url)
        except Exception:
            pass

    return found_urls

# --- 2. BUILD KNOWLEDGE BASE ---
def build_vector_store(urls):
    print(f"\n📥 Loading content from {len(urls)} pages...")
    loader = WebBaseLoader(urls)
    docs = loader.load()

    print("✂️ Splitting text into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(docs)

    print("🧠 Creating embeddings (running locally on Colab)...")
    # This uses a small, free model from Hugging Face to index the data
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore

# --- 3. CHAT FUNCTION ---
def start_chatbot():
    # A. Get URL
    target_url = input("\nEnter the Homepage URL: ")
    urls = crawl_website(target_url)

    if not urls:
        print("❌ No pages found. Check the URL.")
        return

    # B. Index Content
    vectorstore = build_vector_store(urls)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Retrieve top 3 matches

    chat_history = []

    print("\n✅ Chatbot Ready! (Powered by google.colab.ai). Type 'exit' to quit.\n")

    while True:
        user_query = input("You: ")
        if user_query.lower() in ['exit', 'quit']:
            break

        # 1. Retrieve relevant context from the website
        relevant_docs = retriever.invoke(user_query)
        context_text = "\n\n".join([doc.page_content for doc in relevant_docs])

        # 2. Construct the prompt with context
        prompt = f"""You are a helpful assistant. Answer the user's question based strictly on the context provided below.

Context from website:
{context_text}

Chat History:
{chat_history[-3:]}

User Question: {user_query}

Answer:"""

        # 3. Generate Answer using the built-in free API
        try:
            # This is the new built-in function
            response = ai.generate_text(prompt, model_name="gemini-2.0-flash-lite")
            print(f"Gemini: {response}\n")

            chat_history.append(f"User: {user_query}\nAI: {response}")

        except Exception as e:
            print(f"Error generating response: {e}")

# Run the app
start_chatbot()

ModuleNotFoundError: No module named 'langchain.text_splitter'